In [1]:
import argparse
from omegaconf import OmegaConf
import gymnasium
from world_models import WorldModel

import colorama
import numpy as np
from tinygrad import Tensor, nn
from tinygrad.nn.state import get_parameters, get_state_dict

import env_wrapper

In [2]:
def build_single_env(env_name, image_size, seed):
  env = gymnasium.make(env_name, full_action_space=False, render_mode="rgb_array", frameskip=1)
  env = env_wrapper.SeedEnvWrapper(env, seed=seed)
  env = env_wrapper.MaxLast2FrameSkipWrapper(env, skip=4)
  env = gymnasium.wrappers.ResizeObservation(env, shape=image_size)
  env = env_wrapper.LifeLossInfo(env)
  return env

In [3]:
def build_world_model(conf, action_dim):
    return WorldModel(
        in_channels=conf.models.world_model.in_channels,
        action_dim=action_dim,
        transformer_max_length=conf.models.world_model.transformer_max_length,
        transformer_hidden_dim=conf.models.world_model.transformer_hidden_dim,
        transformer_num_layers=conf.models.world_model.transformer_num_layers,
        transformer_num_heads=conf.models.world_model.transformer_num_heads
    )

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument("-n", type=str, required=True)
parser.add_argument("-seed", type=int, required=True)
parser.add_argument("-config_path", type=str, required=True)
parser.add_argument("-env_name", type=str, required=True)
parser.add_argument("-trajectory_path", type=str, required=True)
args = parser.parse_args([
    "-n", "MsPacman",
    "-seed", "1",
    "-config_path", "STORM.yaml",
    "-env_name", "ALE/MsPacman-v5",
    "-trajectory_path", "D_TRAJ/MsPacman.pkl"
])
conf = OmegaConf.load(args.config_path)

In [5]:
print(colorama.Fore.RED + str(args) + colorama.Style.RESET_ALL)

np.random.seed(args.seed)
Tensor.manual_seed(args.seed)

Namespace(n='MsPacman', seed=1, config_path='STORM.yaml', env_name='ALE/MsPacman-v5', trajectory_path='D_TRAJ/MsPacman.pkl')


In [6]:
dummy_env = build_single_env(args.env_name, conf.basic_settings.image_size, seed=0)
action_dim = dummy_env.action_space.n.item()
del dummy_env

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [7]:
world_model = build_world_model(conf, action_dim)

In [8]:
i = 1
updatable_count = 0
for k, v in get_state_dict(world_model).items():
    print(i, "- ", k, ": ", v.shape, " -- ", v.requires_grad)
    if v.requires_grad is None: updatable_count += 1
    i+=1
print("Total updatable parameters: ", updatable_count)

1 -  encoder.backbone.0.weight :  (32, 3, 4, 4)  --  None
2 -  encoder.backbone.1.weight :  (32,)  --  None
3 -  encoder.backbone.1.bias :  (32,)  --  None
4 -  encoder.backbone.1.running_mean :  (32,)  --  False
5 -  encoder.backbone.1.running_var :  (32,)  --  False
6 -  encoder.backbone.1.num_batches_tracked :  (1,)  --  False
7 -  encoder.backbone.3.weight :  (64, 32, 4, 4)  --  None
8 -  encoder.backbone.4.weight :  (64,)  --  None
9 -  encoder.backbone.4.bias :  (64,)  --  None
10 -  encoder.backbone.4.running_mean :  (64,)  --  False
11 -  encoder.backbone.4.running_var :  (64,)  --  False
12 -  encoder.backbone.4.num_batches_tracked :  (1,)  --  False
13 -  encoder.backbone.6.weight :  (128, 64, 4, 4)  --  None
14 -  encoder.backbone.7.weight :  (128,)  --  None
15 -  encoder.backbone.7.bias :  (128,)  --  None
16 -  encoder.backbone.7.running_mean :  (128,)  --  False
17 -  encoder.backbone.7.running_var :  (128,)  --  False
18 -  encoder.backbone.7.num_batches_tracked :  (1,)

# World model safetensors from torch

In [9]:
from safetensors import safe_open

In [10]:
state_dict = {}
with safe_open("world_model.safetensors", framework="pt", device=0) as f:
    for k in f.keys():
        state_dict[k] = f.get_tensor(k)

In [11]:
state_dict.keys()

dict_keys(['dist_head.post_head.bias', 'dist_head.post_head.weight', 'dist_head.prior_head.bias', 'dist_head.prior_head.weight', 'encoder.backbone.0.weight', 'encoder.backbone.1.bias', 'encoder.backbone.1.num_batches_tracked', 'encoder.backbone.1.running_mean', 'encoder.backbone.1.running_var', 'encoder.backbone.1.weight', 'encoder.backbone.10.bias', 'encoder.backbone.10.num_batches_tracked', 'encoder.backbone.10.running_mean', 'encoder.backbone.10.running_var', 'encoder.backbone.10.weight', 'encoder.backbone.3.weight', 'encoder.backbone.4.bias', 'encoder.backbone.4.num_batches_tracked', 'encoder.backbone.4.running_mean', 'encoder.backbone.4.running_var', 'encoder.backbone.4.weight', 'encoder.backbone.6.weight', 'encoder.backbone.7.bias', 'encoder.backbone.7.num_batches_tracked', 'encoder.backbone.7.running_mean', 'encoder.backbone.7.running_var', 'encoder.backbone.7.weight', 'encoder.backbone.9.weight', 'image_decoder.backbone.0.weight', 'image_decoder.backbone.10.weight', 'image_deco

In [17]:
Tensor(state_dict['dist_head.post_head.bias'].cpu().numpy())

'CUDA'

In [ ]:
for k, v in get_state_dict(world_model).items():
    if k in state_dict.keys():
        print(k, " -- ", "yes")
    else:
        print(k, " -- ", "no")

In [24]:
from tqdm import tqdm

model_state_dict = get_state_dict(world_model)
for k, v in tqdm(model_state_dict.items()):
    if k in state_dict.keys():
        if state_dict[k].ndim == 0: t = state_dict[k].reshape(1).cpu().numpy()
        else: t = state_dict[k].cpu().numpy()
        model_state_dict[k].assign(Tensor(t)).realize()

  0%|          | 0/104 [00:00<?, ?it/s]

100%|██████████| 104/104 [00:00<00:00, 1066.24it/s]


In [29]:
world_model_inputs = {}
with safe_open("world_model_inputs.safetensors", framework="pt", device=0) as f:
    for k in f.keys():
        world_model_inputs[k] = f.get_tensor(k)

world_model_inputs = {k: Tensor(v.cpu().numpy()) for k, v in world_model_inputs.items()}

In [37]:
with Tensor.train():
    total_loss = world_model.loss(world_model_inputs["obs"], world_model_inputs["action"], world_model_inputs["reward"], world_model_inputs["termination"],logger=None)

AttributeError: 'WorldModel' object has no attribute 'zero_grad'

In [35]:
world_model_outputs = {}
with safe_open("world_model_outputs.safetensors", framework="pt", device=0) as f:
    for k in f.keys():
        world_model_outputs[k] = f.get_tensor(k)

In [36]:
world_model_outputs["total_loss"]

tensor(6644.0435, device='cuda:0')

In [34]:
total_loss.numpy()

array(6646.534, dtype=float32)